In [ ]:
dataDirectory = "dat/ReArm.lnk/ReArm_C1P02"

# load goodFiles.txt into a list
goodFiles = []
with open("../" + dataDirectory + "/goodFiles.txt", "r") as f:
    for line in f:
        goodFiles.append(line.strip())

print("goodFiles.txt contains {} files".format(len(goodFiles)))

In [ ]:
expectedStreams = [
    ["EuroMov-Markers-Kinect", "Markers"],
    ["NIC-EEG", "EEG"],
    ["NIC-Markers", "Markers"],
    ["NIC-Quality", "Quality"],
    ["EuroMov-Mocap-Kinect", "MoCap"],
    ["Mouse", "Markers"],
    ["MouseToNIC", "Markers"],
    ["Mouse", "MoCap"],
    ["Oxysoft", "NIRS"],
    ["NIC-Accelerometer", "Accelerometer"],
    ["Oxysoft Event", "Event"],
    #["Oxysoft Event", "toto"],
]

# Explore the synchrony of the XDF and CSV files


In [ ]:
%matplotlib qt

import numpy as np
from datetime import datetime
import os
import pyxdf


fullFname = goodFiles[15] # xdf file
#fullFname = goodFiles[0] # csv file

# analyse the extension of the file
fpath = os.path.dirname(fullFname)
fname = os.path.basename(fullFname)
fname, extension = os.path.splitext(fname)

# split the filename into tokens
tokens = fname.split("_")


# List all the streams in the XDF file

In [ ]:
# load the full xdf file to show the streams
data, header = pyxdf.load_xdf(
    fullFname, synchronize_clocks=True, dejitter_timestamps=False
)
# list all the streams in the file
print("Found {} streams:".format(len(data)))
for ix, stream in enumerate(data):
    print(
        "  {:02d}: {} {} - {} - shape {} at {} Hz (effective {:5.2f} Hz)".format(
            stream["info"]["stream_id"],
            ix + 1,
            stream["info"]["name"][0],
            stream["info"]["type"][0],
            (int(stream["info"]["channel_count"][0]), len(stream["time_stamps"])),
            stream["info"]["nominal_srate"][0],
            stream["info"]["effective_srate"],
        )
    )

# check if all the expected streams are present
foundStreams = []
missingStreams = []
for i in range(len(expectedStreams)):
    expectedStream = expectedStreams[i]
    found = False
    for ix, stream in enumerate(data):
        if stream["info"]["name"][0] == expectedStream[0] and stream["info"]["type"][0] == expectedStream[1]:
            found = True
            foundStreams.append(expectedStream)
    if not found:
        missingStreams.append(expectedStream)

# print the missing streams
print("Missing streams:")
if len(missingStreams) == 0:
    print("  None")
else:
    for stream in missingStreams:
        print("  {}".format(stream))

# print the found streams
print("Found streams:")
for stream in foundStreams:
    print("  {}".format(stream))

# Explore the synchrony of the XDF and CSV files for the Kinect data 

The CSV and XDF files contain the exact same information, but the XDF file has timestamps relative to the start of the kinect recording, while the CSV file has absolute timestamps (i.e., from the host computer).
If the CSV to XDF delay is constant, this can be used as a correction factor for the (wrong) timestamps of the kinect stream.

In [ ]:
# load the EuroMov-Mocap-Kinect stream
data, header = pyxdf.load_xdf(
    filename=fullFname,
    select_streams=[{"type": "MoCap", "name": "EuroMov-Mocap-Kinect"}],
    synchronize_clocks=True,
    dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
)
data_XDF = data[0]

# get the corresponding kinect file name
kinectFname = (
    tokens[0]
    + "_"
    + tokens[1]
    + "_"
    + tokens[2]
    + "_"
    + tokens[3]
    + "_"
    + tokens[4]
    + "_k.csv"
)
fullFname_csv = os.path.join(fpath, kinectFname)

# get the corresponding data from the CSV file
data_csv = np.loadtxt(fullFname_csv, skiprows=3, delimiter=",", dtype=float)

# get the timestamps from the two files
timestampCSV_sec = data_csv[:, 0] / 1000
timestampXDF_sec = data_XDF["time_stamps"]

# get the (median) delay between the two files
delay = np.median(timestampXDF_sec - timestampCSV_sec)

# get the distribution of the delays
delays = timestampXDF_sec - timestampCSV_sec - delay


In [ ]:
# do the plots
import matplotlib.pyplot as plt

# make a boxplot of the distribution of the delays
fig = plt.figure()
plt.boxplot(delays, vert=False)
plt.xlabel("Delay (s)")
# add a title with the median delay
plt.title("Distribution of the delays")
plt.show()

# make a plot of the timestampXDF and timestampCSV
fig = plt.figure()
plt.plot(timestampXDF_sec - timestampXDF_sec[0], "o", label="XDF")
plt.plot(timestampCSV_sec - timestampCSV_sec[0], "o", label="CSV")
plt.ylabel("TimeStamp (s)")
plt.xlabel("Line number")
plt.legend()
plt.show()

# make a plot of the distribution of the delays
fig = plt.figure()
plt.plot(delays * 1000, "o")
plt.ylabel("Delay (milliseconds)")
plt.xlabel("Line number")
plt.show()

## Print the info about each stream 

In [ ]:
import os
import pyxdf
import numpy as np
import matplotlib.pyplot as plt


def print_all_streamInfo(data):
    """
    Prints the info of each stream in the XDF file.
    """

    shiftBy = "  "

    for stream in data:
        print(stream["info"]["name"][0])
        for k, v in stream["info"].items():
            printList(shiftBy, k, v)


def printList(shift, name, myList):
    """
    Prints a list (of array of list) with str items.
    The function recurs until it finds a leaf that is a str.
    Each recurrence is shifted to account for the structure.
    """

    def printLeaf(shift, name, item):
        print(shift, name, item)

    if isinstance(myList, list):
        for i in range(0, len(myList)):
            if isinstance(myList[i], str):
                printLeaf(shift, name, myList[i])
            else:
                name_i = name + "[" + str(i) + "]"
                print(shift, name_i)
                if myList[i] is not None:
                    for vi, ki in myList[i].items():
                        printList(shift + shift, vi, ki)
    else:
        printLeaf(shift, name, myList)


####################  MAIN  ####################

fullFname = goodFiles[8]  # xdf file
data, header = pyxdf.load_xdf(fullFname)
print_all_streamInfo(data)